In [27]:
!echo "Last updated: `date`"

Last updated: Tue Aug  4 17:03:45 PDT 2020


## Model

$$
\begin{eqnarray}
y_i \mid z_i &\sim& \text{Bernoulli}(\text{sigmoid}(z_i)) \\
\mathbf{z} \mid \rho, \alpha &\sim& \text{MvNormal}(\mathbf{0}, K_{\alpha, \rho}(\mathbf{X})) \\
\alpha &\sim& \text{LogNormal}(0, 0.1) \\
\rho &\sim& \text{LogNormal}(0, 1) \\
\end{eqnarray}
$$

**Note:** The latent GP should be centered at 0, because when there is no data, $P(y_i = 1)$ should be $0.5$.

In [4]:
import json
import numpy as np
import plotly.graph_objects as go

In [2]:
np.random.seed(1)

In [21]:
# Read data.
path_to_data = '../data/gp-binary-data-N100.json'
simdata = json.load(open(path_to_data))

# Store data.
X = np.stack([simdata['x1'], simdata['x2']], axis=-1)
y = np.array(simdata['y'])

In [25]:
def plot_data(X, y):
    colors = np.array(['red', 'blue'])
    go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers',
                             marker=dict(color=colors[y])))
    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20)
    )
    fig.show()

In [26]:
plot_data(X, y)